### タスク 1.1: 基盤セットアップと基本エージェントの作成

Strands Agentsフレームワークを使用してカスタマーサポートエージェントのプロトタイプをセットアップします。このプロトタイプは、エージェントのプロトタイプから本番環境対応ソリューションまでの完全な道のりを探索するための出発点となります。

このタスクを完了すると、エージェントは次の基本アーキテクチャを持つようになります:

<div style="text-align:left">
    <img src="images/architecture_lab1_strands.png" width="75%"/>
</div>

*画像の説明: ローカルツールを使用してローカルで実行されるシンプルなエージェントプロトタイプ*

開発環境を準備するために、AWS SDK、AgentCoreコンポーネント、Strandsフレームワークを含むすべての必要なライブラリの依存関係をインストールしてインポートします。

In [ ]:
import boto3
import json
import uuid
import time
import requests
from boto3.session import Session

# AgentCoreのインポート 
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

# Strandsのインポート 
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry, MessageAddedEvent
from mcp.client.streamable_http import streamablehttp_client

# ローカルツール 
from lab_helpers.lab1_strands_agent import (
    get_product_info, get_return_policy, get_technical_support, web_search,
    SYSTEM_PROMPT, MODEL_ID
)
from lab_helpers.utils import get_ssm_parameter, put_ssm_parameter
from scripts.utils import get_cognito_client_secret

# セットアップ 
boto_session = Session()
REGION = boto_session.region_name
CUSTOMER_ID = "customer_001"
SESSION_ID = str(uuid.uuid4())

print("✅ Libraries imported successfully!")

エージェントを作成する前に、カスタマーサポート機能を強化するローカルツールを確認します。`lab_helpers/lab1_strands_agent.py`を開いて確認してください:

- ツールは`@tool`デコレータを使用してこのファイルでローカルに定義されています
- 4つのツール関数とその目的:
  - get_product_info(): 製品情報を取得
  - get_return_policy(): 特定の製品の返品ポリシーを取得
  - get_technical_support(): 技術サポートガイダンスを提供
  - web_search(): 最新情報をウェブで検索
- モックデータの使用方法(実際のデータベース/APIをシミュレート)
- エージェントの動作を定義するシステムプロンプト

クエリの理解からアクションの実行まで、コアAI機能を実証する基本的なカスタマーサポートエージェントを作成します。このエージェントは以下を組み合わせます:

- **基盤モデル**: 推論と意思決定を支える「頭脳」
- **システムプロンプト**: エージェントの性格とサービス基準を定義する行動指示
- **専門ツール**: 4つのローカルツール(製品情報、返品ポリシー、技術サポート、ウェブ検索)

エージェントを呼び出すと、次のプロセスに従います:
1. **クエリ分析**: エージェントが顧客の質問を分析
2. **ツール選択**: エージェントが使用するツールを決定(必要に応じて)
3. **ツール実行**: エージェントが適切なパラメータで適切なツールを呼び出し
4. **応答合成**: エージェントがツールの結果と知識を組み合わせて有用な応答を作成
5. **品質チェック**: エージェントがシステムプロンプトの基準を満たしているか確認

In [ ]:
# ローカルツールを使用した基本エージェントを作成 
model = BedrockModel(model_id=MODEL_ID, temperature=0.3, region_name=REGION)
basic_agent = Agent(
    model=model,
    tools=[get_product_info, get_return_policy, get_technical_support, web_search],
    system_prompt=SYSTEM_PROMPT
)

print("✅ Basic customer support agent ready!")
print("📋 Available tools: Product Info, Return Policy, Technical Support, Web Search")

基本エージェントをテストして、顧客のクエリをどのように処理し、ツールを使用するかを確認します:

In [ ]:
# 基本エージェント機能をテスト 
print("💬 Testing basic agent...\n")
response = basic_agent("Laptopの返品ポリシーは何ですか?")
print("\n" + "="*50 + "\n")

この初期プロトタイプには、後続のタスクで対処するいくつかの制限があります:

- **永続的なメモリがない** - エージェントは以前のセッションからの顧客履歴と設定を忘れる
- **ローカルツールのみ** - 共有またはエンタープライズグレードのツール統合がない
- **アイデンティティ管理がない** - 特定のユーザーの代わりに行動できない

### タスク 1.2: メモリでエージェントを強化

大切な顧客が最近の注文に関する問題についてサポートチームに連絡します。彼らは自分の好みを説明し、不満を共有し、エージェントと協力して問題を解決します。3週間後、関連する質問で再度連絡します。しかし今度は、すべてを繰り返さなければなりません - 好み、履歴、コンテキスト - エージェントは現在の会話セッションのみを記憶し、以前のセッションは記憶していないためです。これにより以下が発生します:
- **不満を持つ顧客** - 情報を繰り返し繰り返さなければならない
- **非効率的なサポート** - 以前のやり取りに基づいて構築できない
- **顧客満足度の低下** - 非個人的で一般的な応答のため

Amazon Bedrock AgentCore Memoryは、AIエージェントが時間の経過とともにコンテキストを維持し、重要な事実を記憶し、一貫性のある個別化された体験を提供できるようにするマネージドサービスを提供することで、この制限に対処します。AgentCore Memoryは2つのレベルで動作します:
- **短期メモリ**: 即座の会話コンテキストとセッションベースの情報(Strands Agentフレームワークによって自動的に処理)
- **長期メモリ**: 複数の会話にわたって抽出された永続的な情報(事実、好み、要約を含む)(USER_PREFERENCEおよびSEMANTIC戦略を使用したAgentCore Memoryサービスを通じて実装)

プロトタイプを、次の例のような機能を持つ顧客対応アシスタントに変換します:
- **「お帰りなさい、Sarah!」** - 戻ってきた顧客を即座に認識
- **「先月のラップトップの問題についてフォローアップ」** - 関連する会話をシームレスに接続
- **「購入履歴に基づいて、これをお勧めします」** - 個別化された提案を提供

このタスクを完了すると、エージェントは統合されたメモリ機能を持つ次のアーキテクチャを持つようになります:

<div style="text-align:left">
    <img src="images/architecture_lab2_memory.png" width="75%"/>
</div>

*画像の説明: 永続的な顧客コンテキストと個別化のためのAgentCore Memoryで強化されたエージェント*

**メモリ戦略の構成**: 2つのインテリジェント戦略を組み合わせたメモリリソースを作成します:

| 戦略タイプ | 目的 | 顧客へのメリット |
|---------------|---------|------------------|
| USER_PREFERENCE | 顧客の好みと行動 | 「あなたが好むことを覚えています...」 |
| SEMANTIC | 事実情報とコンテキスト | 「以前の問題について...」 |

AgentCore Memoryは、actorIdを使用して長期メモリメッセージを論理的にグループ化するために名前空間を使用します:
- `support/customer/{actorId}/preferences`: ユーザー設定メモリ戦略用
- `support/customer/{actorId}/semantic`: セマンティックメモリ戦略用

In [ ]:
# AgentCore Memoryサービス用のメモリクライアントを初期化 
memory_client = MemoryClient(region_name=REGION)
memory_name = "CustomerSupportMemory"

def create_or_get_memory_resource():
    try:
        # SSMパラメータから既存のメモリリソースの取得を試行 
        memory_id = get_ssm_parameter("/app/customersupport/agentcore/memory_id")
        memory_client.gmcp_client.get_memory(memoryId=memory_id)
        return memory_id
    except:
        # 2つの戦略で新しいメモリリソースを作成 
        strategies = [
            {
                # USER_PREFERENCE戦略は顧客の好みと行動をキャプチャ 
                StrategyType.USER_PREFERENCE.value: {
                    "name": "CustomerPreferences",
                    "description": "Captures customer preferences and behavior",
                    "namespaces": ["support/customer/{actorId}/preferences"],
                }
            },
            {
                # SEMANTIC戦略は会話からの事実情報を保存 
                StrategyType.SEMANTIC.value: {
                    "name": "CustomerSupportSemantic",
                    "description": "Stores facts from conversations",
                    "namespaces": ["support/customer/{actorId}/semantic"],
                }
            },
        ]
        print("Creating AgentCore Memory resources (2-3 minutes)...")
        # メモリリソースを作成して完了を待機 
        response = memory_client.create_memory_and_wait(
            name=memory_name,
            description="Customer support agent memory",
            strategies=strategies,
            event_expiry_days=90,  # メモリイベントは90日後に期限切れ 
        )
        memory_id = response["id"]
        # 将来の使用のためにメモリIDをSSMに保存 
        put_ssm_parameter("/app/customersupport/agentcore/memory_id", memory_id)
        return memory_id

memory_id = create_or_get_memory_resource()
print(f"✅ Memory resource ready: {memory_id}")

「customer_001」という名前の戻ってきた顧客をシミュレートします。この顧客は以前にサポートチームとやり取りしたことがあります。これは、AgentCore Memoryが個々の会話を豊かで永続的な顧客インサイトに自動的に変換する方法を示します。以前の顧客のやり取りを読み込み、AgentCore Memoryが自動的にそれらを長期的な顧客インサイトに変換するのを見てください。

In [ ]:
# 以前の顧客のやり取りをシード 
previous_interactions = [
    ("ビデオ編集中にMacBook Proが過熱する問題が発生しています.", "USER"),
    ("熱の問題についてはお手伝いできます。ご注文番号MB-78432のMacBook Proはまだ保証期間内です。", "ASSISTANT"),
    ("ゲーミングヘッドフォンの返品ポリシーは？FPSの対戦ゲームには低遅延が必要です", "USER"),
    ("ゲーミングヘッドフォンは30日間返品可能です。FPSの競技志向なので、オーディオレイテンシーの仕様を確認することをお勧めします。", "ASSISTANT"),
    ("プログラミング用に1200ドル以下のノートパソコンを探しています。最低16GBのRAMがあり、Linuxとの互換性が良いものが望ましいです。ThinkPadモデルが好みです。", "USER"),
    ("完璧です！開発作業には、Linux サポートが優れた ThinkPad E シリーズまたは Dell XPS モデルをお勧めします。", "ASSISTANT"),
]

if memory_id:
    memory_client.create_event(
        memory_id=memory_id,
        actor_id=CUSTOMER_ID,
        session_id="previous_session",
        messages=previous_interactions
    )
    print("✅ Customer history seeded successfully")
    print("⏳ Long-term memory processing will begin automatically...")

Strands Agentsは、強く型付けされたイベントコールバックを通じてコンポーネントがエージェントの動作に反応または変更できるようにする強力なフックシステムを提供します。これにより、手動介入なしでメモリ操作が自動的に行われることが保証されます。

顧客がエージェントとやり取りするたびに、自動的に以下が行われます:
- **以前のやり取りと好みに基づいて会話を個別化**
- **新しいやり取りをメモリに追加**して、将来の個別化を継続的に改善

フック統合が行うこと:
- **応答前**: 関連する顧客コンテキストと好みを自動的に取得
- **応答後**: 新しいやり取りをAgentCore Memoryに自動的に保存

自動顧客コンテキストをメモリフックで有効にします。

In [ ]:
class CustomerSupportMemoryHooks(HookProvider):
    def __init__(self, memory_id: str, client: MemoryClient, actor_id: str, session_id: str):
        self.memory_id = memory_id
        self.client = client
        self.actor_id = actor_id
        self.session_id = session_id
        self.namespaces = {
            i["type"]: i["namespaces"][0]
            for i in self.client.get_memory_strategies(self.memory_id)
        }

    def retrieve_customer_context(self, event: MessageAddedEvent):
        # エージェントが応答する前に実行されるフック（顧客コンテキストを取得） 
        messages = event.agent.messages
        # ユーザーメッセージのみを処理（ツール結果は除く） 
        if messages[-1]["role"] == "user" and "toolResult" not in messages[-1]["content"][0]:
            user_query = messages[-1]["content"][0]["text"]
            
            try:
                all_context = []
                # 各戦略名前空間からメモリを取得（USER_PREFERENCEとSEMANTICの両方） 
                for context_type, namespace in self.namespaces.items():
                    memories = self.client.retrieve_memories(
                        memory_id=self.memory_id,
                        namespace=namespace.format(actorId=self.actor_id),
                        query=user_query,
                        top_k=3,  # 関連性の高いメモリ上位3件を取得 
                    )
                    # メモリオブジェクトからテキストコンテンツを抽出 
                    for memory in memories:
                        if isinstance(memory, dict):
                            content = memory.get("content", {})
                            if isinstance(content, dict):
                                text = content.get("text", "").strip()
                                if text:
                                    all_context.append(f"[{context_type.upper()}] {text}")
                
                # ユーザーメッセージに顧客コンテキストを前置 
                if all_context:
                    context_text = "\n".join(all_context)
                    original_text = messages[-1]["content"][0]["text"]
                    messages[-1]["content"][0]["text"] = f"Customer Context:\n{context_text}\n\n{original_text}"
            except Exception as e:
                print(f"Failed to retrieve customer context: {e}")

    def save_support_interaction(self, event: AfterInvocationEvent):
        # エージェントが応答した後に実行されるフック（やり取りをメモリに保存） 
        try:
            messages = event.agent.messages
            # ユーザーとアシスタントの両方のメッセージがある場合のみ保存 
            if len(messages) >= 2 and messages[-1]["role"] == "assistant":
                customer_query = None
                agent_response = None
                
                # 最新のユーザークエリとアシスタント応答を検索 
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not agent_response:
                        agent_response = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not customer_query and "toolResult" not in msg["content"][0]:
                        customer_query = msg["content"][0]["text"]
                        break
                
                # やり取りをAgentCore Memoryに保存 
                if customer_query and agent_response:
                    self.client.create_event(
                        memory_id=self.memory_id,
                        actor_id=self.actor_id,
                        session_id=self.session_id,
                        messages=[(customer_query, "USER"), (agent_response, "ASSISTANT")],
                    )
        except Exception as e:
            print(f"Failed to save support interaction: {e}")

    def register_hooks(self, registry: HookRegistry) -> None:
        # 両方のフックをエージェントのフックレジストリに登録 
        registry.add_callback(MessageAddedEvent, self.retrieve_customer_context)
        registry.add_callback(AfterInvocationEvent, self.save_support_interaction)

print("✅ Memory hooks defined - Automatic customer personalization enabled!")
print("🧠 Your agent will now remember customers and personalize every interaction")

メモリ強化エージェントを作成してテストし、顧客コンテキストを取得して応答を個別化する方法を確認します:

In [ ]:
# フック付きのメモリ強化エージェントを作成 
memory_hooks = CustomerSupportMemoryHooks(memory_id, memory_client, CUSTOMER_ID, SESSION_ID)

memory_agent = Agent(
    model=model,
    tools=[get_product_info, get_return_policy, get_technical_support, web_search],
    hooks=[memory_hooks],
    system_prompt=SYSTEM_PROMPT
)

print("✅ Memory-enhanced agent created!")
print("🧠 Agent will automatically retrieve customer context and save interactions")

In [ ]:
# メモリ処理の完了を待機 
print("⏳ Waiting 60 seconds for memory processing to complete...")
time.sleep(60)

# メモリ呼び出しをテスト 
print("🧠 Testing memory-enhanced agent...\n")
response = memory_agent("私のノートパソコンの好みは何ですか?")
print("\n" + "="*50 + "\n")

### タスク 1.3: ゲートウェイ統合とAgentCore Identityでスケール

メモリが配置されたら、強力なツールに焦点を当て、その影響をスケールします。優れたエージェントには、独自およびサードパーティのAPIとデータを最大限に活用するツールが必要であり、これにより、これらのエージェントは内部および外部の顧客の両方のために物事を成し遂げることができます。ただし、エージェントツールの構築、保護、スケーリングは困難であり、顧客がエージェントのプロトタイプから本番環境での実際のエージェントビジネス価値への移行を妨げる重大な障害となります。AgentCore Gatewayは、AIエージェントが統一された**Model Context Protocol (MCP)**エンドポイントを通じて実世界のツールを発見、認証、呼び出すことを可能にする接続レイヤーとして機能します。これは、数百のAPI、リソース、ツールを管理する企業にとって重要です。

主な利点:
- **完全管理されたMCPサーバー**ソリューション(インフラストラクチャ管理なし)
- **既存のAPIの統合**とLambda関数
- **多様なツール間の統一インターフェース**
- **安全な認証**と承認
- **セマンティックツールの発見**と選択

##### 構築するもの:

**ツールの一元化と再利用性:**
- ウェブ検索をローカルツールから一元化されたAgentCore Gatewayに移行
- 既存のエンタープライズLambda関数(保証チェック)を統合
- 複数のエージェントタイプがアクセスできる共有ツールインフラストラクチャを作成

**エンタープライズグレードのセキュリティ:**
- Cognito統合によるJWTベースの認証を実装
- ゲートウェイアクセスのための安全なインバウンド承認を構成
- ツール使用のためのアイデンティティベースのアクセス制御を確立

これにより、ツールが一元管理され、複数のエージェントタイプで再利用可能なスケーラブルな基盤が作成され、コードの重複が排除され、メンテナンスが簡素化されます。

AgentCore Identityもこのプロセスに関与しています。これにより、AIエージェントがAWSリソースに安全にアクセスでき、Amazon Cognitoと連携してインバウンド呼び出し元認証を処理するのに役立ちます。また、AIエージェントがアウトバウンド認証を使用してサードパーティのツールとサービスに安全にアクセスできるようにしますが、このAgentcore Identityの機能はこのラボでは使用されません。

<div style="text-align:left">
    <img src="images/architecture_lab3_identity.png" width="75%"/>
</div>

このタスクを完了すると、エージェントは統合されたゲートウェイ機能を持つ次のアーキテクチャを持つようになります:

<div style="text-align:left">
    <img src="images/architecture_lab3_gateway.png" width="75%"/>
</div>

*画像の説明: 安全で一元化されたツール管理とエンタープライズ統合のためのAgentCore Gatewayで強化されたエージェント*

Lambda関数をMCP互換エンドポイントとして公開するAgentCore Gatewayを作成します。ツールを呼び出す権限のある呼び出し元を検証するために、OAuth承認(MCPサーバーの標準)を使用して**インバウンド認証**を構成します。

### ゲートウェイ認証の理解

AgentCore Gatewayは**OAuth 2.0とJWTトークン**を使用してツールへのアクセスを保護します。これにより、権限のないアプリケーションがLambda関数を呼び出すことを防ぎます。

**主要な概念:**

1. **認証プロバイダー**: Amazon Cognitoがアイデンティティを管理し、トークンを発行
2. **クライアント認証情報**: エージェントはclient_idとclient_secret(アプリケーションのユーザー名/パスワードのようなもの)を使用
3. **JWTトークン**: エージェントが承認されていることを証明する短期間のトークン
4. **許可されたクライアント**: ゲートウェイにアクセスできるクライアントIDの許可リスト

**仕組み:**
```
エージェント → Cognito: 「これが私のclient_idとclient_secretです」
Cognito → エージェント: 「これがあなたのJWTアクセストークンです」
エージェント → ゲートウェイ: 「これが私のトークンです」
ゲートウェイ → Cognito: 「このトークンは有効で、許可されたクライアントからのものですか?」
ゲートウェイ → エージェント: 「アクセスが許可されました」
```

**セキュリティ注意**: これらの認証情報は事前に作成され、SSM Parameter Storeに安全に保存されています。認証情報はコードにハードコーディングしないでください

In [ ]:
# SSM Parameter Storeから認証構成を取得 
# これらの値はCloudFormationテンプレートによって作成されました 

# クライアントID: リクエストを行っているアプリケーションを識別 
machine_client_id = get_ssm_parameter("/app/customersupport/agentcore/machine_client_id")
print(f"Machine Client ID: {machine_client_id}")

# ディスカバリURL: CognitoのOAuth構成を見つける場所をゲートウェイに伝える 
# このURLはトークンエンドポイント、サポートされているスコープなどに関するメタデータを提供 
cognito_discovery_url = get_ssm_parameter("/app/customersupport/agentcore/cognito_discovery_url")
print(f"Discovery URL: {cognito_discovery_url}")

# ゲートウェイのJWTベース認証を構成 
auth_config = {
    "customJWTAuthorizer": {
        # このクライアントIDからのトークンのみが受け入れられます 
        "allowedClients": [machine_client_id],
        # ゲートウェイはこのURLからOAuthメタデータを取得 
        "discoveryUrl": cognito_discovery_url
    }
}

print("✅ Authentication configuration ready")

### AgentCore Gatewayの作成

ゲートウェイは、エージェントとバックエンドLambda関数の間の安全なプロキシとして機能します。AIエージェント専用に設計されたAPIゲートウェイと考えてください。

**作成するもの:**
- **ゲートウェイインフラストラクチャ**: コアゲートウェイリソース
- **MCPプロトコル**: ツール通信の標準プロトコル
- **JWT承認**: 先ほど作成した認証構成を使用
- **IAMロール**: Lambda関数を呼び出す権限

**次に起こること:**
1. ゲートウェイを作成(このセル)
2. ツール定義を使用してLambdaターゲットを追加(次のセル)
   - 1つのLambda関数が複数のツールを処理: `check_warranty_status`と`web_search`
   - ゲートウェイはツール名をLambdaに渡し、Lambdaが適切なハンドラーにルーティング
3. エージェントをゲートウェイに接続

**注意**: CloudFormationテンプレートは、複数のツール操作を処理できる単一のLambda関数(`CustomerSupportLambda`)をデプロイしました。これは、各ツールに個別のLambda関数をデプロイするよりも効率的です。

In [ ]:
class CreationFailedError(Exception):
    def __init__(self, message):
        self.message = message
        super().__init__(self.message)

gateway_client = boto3.client("bedrock-agentcore-control", region_name=REGION)
gateway_name = "customersupport-gw"

try:
    print(f"Creating gateway: {gateway_name}")
    create_response = gateway_client.create_gateway(
        name=gateway_name,
        roleArn=get_ssm_parameter("/app/customersupport/agentcore/gateway_iam_role"),
        protocolType="MCP",  # Model Context Protocol 
        authorizerType="CUSTOM_JWT",  # 認証にJWTトークンを使用 
        authorizerConfiguration=auth_config,  # 上記の認証構成 
        description="Customer Support AgentCore Gateway",
    )
    gateway_id = create_response["gatewayId"]
    gateway_url = create_response["gatewayUrl"]
    put_ssm_parameter("/app/customersupport/agentcore/gateway_id", gateway_id)

    # ゲートウェイの準備完了を待機 
    print("Waiting for Gateway to be ready...")
    while True:
        status = gateway_client.get_gateway(gatewayIdentifier=gateway_id)['status']
        if status == 'READY':
            break
        elif status == 'FAILED':
            raise CreationFailedError("Gateway creation failed")
        else:
            print(f"  Status: {status}")
            time.sleep(5)

    print(f"✅ Gateway created successfully!")
    print(f"   Gateway ID: {gateway_id}")
    print(f"   Gateway URL: {gateway_url}")
    
except gateway_client.exceptions.ConflictException:
    # ゲートウェイは既に存在するため、取得 
    gateway_id = get_ssm_parameter("/app/customersupport/agentcore/gateway_id")
    gateway_response = gateway_client.get_gateway(gatewayIdentifier=gateway_id)
    gateway_url = gateway_response["gatewayUrl"]
    print(f"✅ Using existing gateway: {gateway_id}")
    
except CreationFailedError:
    print("\033[31m❌ Gateway creation failed. Check CloudWatch logs for details.\033[0m")

Lambda関数をゲートウェイに接続し、エージェントが使用できる2つのツールを定義します:

1. **check_warranty_status**: 製品の保証情報を確認
   - 入力: order_id(文字列)
   - 出力: 保証ステータス、有効期限、カバレッジの詳細

2. **web_search**: 最新情報をウェブで検索
   - 入力: query(文字列)
   - 出力: 検索結果とスニペット

**重要な概念:**
- **単一Lambda、複数ツール**: 1つのLambda関数が両方のツールリクエストを処理
- **ツールルーティング**: ゲートウェイは`toolName`をLambdaに渡し、Lambdaが適切なロジックにルーティング
- **スキーマ定義**: 各ツールには、エージェントが正しく使用する方法を理解するための明確な入力/出力スキーマがある

In [ ]:
# LambdaツールのAPI仕様を読み込み 
api_spec = [
    {
        "name": "check_warranty_status",
        "description": "シリアル番号とメールを使用して保証状況を確認する",
        "inputSchema": {
            "type": "object",
            "properties": {
                "serial_number": {"type": "string"},
                "customer_email": {"type": "string"}
            },
            "required": ["serial_number"]
        }
    },
    {
        "name": "web_search",
        "description": "最新情報をウェブで検索する",
        "inputSchema": {
            "type": "object",
            "properties": {
                "keywords": {"type": "string", "description": "検索クエリキーワード"},
                "region": {"type": "string", "description": "検索リージョン (e.g., us-en)"},
                "max_results": {"type": "integer", "description": "最大の結果"}
            },
            "required": ["keywords"]
        }
    }
]

# ゲートウェイターゲットを作成 
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": get_ssm_parameter("/app/customersupport/agentcore/lambda_arn"),
            "toolSchema": {"inlinePayload": api_spec},
        }
    }
}

try:
    create_target_response = gateway_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name="LambdaTarget",
        description="Lambda tools for customer support",
        targetConfiguration=lambda_target_config,
        credentialProviderConfigurations=[{"credentialProviderType": "GATEWAY_IAM_ROLE"}],
    )
    print(f"✅ Gateway target created: {create_target_response['targetId']}")
except Exception as e:
    print(f"Gateway target may already exist: {str(e)}")

ゲートウェイに接続してツールを使用するようにエージェントを構成します:

**認証フロー:**
1. Cognitoからアクセストークンを取得(client_idとclient_secretを使用)
2. すべてのゲートウェイリクエストにトークンを含める
3. ゲートウェイがトークンを検証し、ツールへのアクセスを許可

**MCP統合:**
- Strands AgentsはModel Context Protocol(MCP)をネイティブにサポート
- MCPClientはゲートウェイへの接続を処理
- エージェントはゲートウェイツールをローカルツールと同じように使用

In [ ]:
def get_cognito_client_secret():
    # Cognito APIを使用してCognitoクライアントシークレットを取得 
    client = boto3.client("cognito-idp")
    response = client.describe_user_pool_client(
        UserPoolId=get_ssm_parameter("/app/customersupport/agentcore/userpool_id"),
        ClientId=get_ssm_parameter("/app/customersupport/agentcore/machine_client_id"),
    )
    return response["UserPoolClient"]["ClientSecret"]

def get_oauth_token():
    # クライアント資格情報フローを使用してゲートウェイ認証用のOAuthトークンを取得 
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",  # OAuth 2.0クライアント資格情報フロー 
        "client_id": get_ssm_parameter("/app/customersupport/agentcore/machine_client_id"),
        "client_secret": get_cognito_client_secret(),
        "scope": get_ssm_parameter("/app/customersupport/agentcore/cognito_auth_scope"),
    }
    # Cognitoからアクセストークンをリクエスト 
    response = requests.post(
        get_ssm_parameter("/app/customersupport/agentcore/cognito_token_url"),
        headers=headers, data=data
    )
    return response.json()

# OAuthアクセストークン（JWT形式）を取得 
token_response = get_oauth_token()
access_token = token_response['access_token']

# Bearerトークン認証を使用してMCPクライアントを作成 
mcp_client = MCPClient(
    lambda: streamablehttp_client(
        gateway_url,
        headers={"Authorization": f"Bearer {access_token}"},  # AuthorizationヘッダーのJWTトークン 
    )
)

print(f"✅ MCP client configured for gateway: {gateway_url}")

ゲートウェイ統合エージェントを作成してテストします:

**アーキテクチャの変更:**
- **以前**: すべてのツールがローカルで定義
- **現在**: 
  - ローカルツール: get_product_info、get_return_policy、get_technical_support
  - ゲートウェイツール: check_warranty_status、web_search

**利点:**
- ツールの一元管理
- 複数のエージェント間での再利用
- エンタープライズセキュリティとスケーリング
- 既存のLambda関数との統合

In [ ]:
# 顧客コンテキスト用のメモリフックを初期化 
memory_hooks = CustomerSupportMemoryHooks(memory_id, memory_client, CUSTOMER_ID, SESSION_ID)

# ゲートウェイへのMCPクライアント接続を開始 
mcp_client.start()
# ゲートウェイから利用可能なツールを取得 
gateway_tools = mcp_client.list_tools_sync()

# ローカルツールと一元化されたゲートウェイツールを結合 
all_tools = [
    get_product_info,      # ローカルツール 
    get_return_policy,     # ローカルツール 
    get_technical_support, # ローカルツール 
] + gateway_tools          # ゲートウェイツール - web_search、check_warranty_status 

# メモリとゲートウェイ統合を備えた強化エージェントを作成 
enhanced_agent = Agent(
    model=model,
    tools=all_tools,           # ローカル + ゲートウェイツール 
    hooks=[memory_hooks],      # 自動メモリ操作 
    system_prompt=SYSTEM_PROMPT
)

print("✅ Enhanced Customer Support Agent created!")
print(f"📊 Total tools available: {len(all_tools)}")
print(f"🧠 Memory enabled with ID: {memory_id}")
print(f"🔒 Secure gateway integration: {gateway_url}")

### 次のステップ

AgentCore Gatewayの基本を習得しました。本番環境では、次のことを検討してください:

**スケーリングとガバナンス:**
- 複数のLambda関数とAPIを単一のゲートウェイに追加
- ツールの使用状況とパフォーマンスを監視
- ツールアクセスのためのきめ細かい権限を実装

**高度な機能:**
- サードパーティAPIとの統合(AgentCore Identity outbound authを使用)
- カスタムツール検証とエラー処理
- ツール使用のレート制限とクォータ

**エンタープライズパターン:**
- 異なる環境(dev/staging/prod)用の複数のゲートウェイ
- ツール定義の集中管理
- 自動ツールドキュメント生成

In [ ]:
# ゲートウェイツールをテスト 
print("🔍 Testing gateway web search...\n")
response2 = enhanced_agent("最新のiPhone 15のトラブルシューティングのヒントを検索して下さい。")
print("\n" + "="*50 + "\n")

In [ ]:
# 保証チェックをテスト 
print("🛡️ Testing warranty check...\n")
response3 = enhanced_agent("シリアル番号ABC12345678の保証状況を確認してください。")
print("\n" + "="*50 + "\n")

In [ ]:
# 組み合わせた機能をテスト
print("🎯 Testing combined memory + gateway capabilities...\n")
response4 = enhanced_agent("ゲーミングヘッドフォンがまた必要になったので、最新のレビューも探して下さい。")
print("\n" + "="*50 + "\n")

### まとめ

このラボでは、シンプルなプロトタイプから本番環境対応のAIエージェントへの道のりを完了しました:

**タスク 1.1 - 基盤**
- Strands Agentsを使用した基本的なカスタマーサポートエージェントを作成
- ローカルツールと基盤モデルの統合を理解

**タスク 1.2 - メモリ**
- AgentCore Memoryで永続的な顧客コンテキストを追加
- 自動メモリ管理のためのフックを実装
- 個別化された顧客体験を実現

**タスク 1.3 - ゲートウェイとアイデンティティ**
- AgentCore Gatewayで一元化されたツール管理を作成
- JWT認証でエンタープライズセキュリティを実装
- 既存のLambda関数を統合

**主要な学習:**
- **プロトタイプから本番環境へ**: ローカル開発からエンタープライズデプロイメントへの明確な道筋
- **マネージドサービス**: AgentCoreコンポーネントがインフラストラクチャの複雑さを処理
- **スケーラブルなアーキテクチャ**: 一元化されたツールとメモリにより、複数のエージェントタイプをサポート
- **エンタープライズ対応**: セキュリティ、認証、統合が組み込まれている

**次のステップ:**
- 追加のツールとAPIを統合
- 複数のエージェントタイプを実装(営業、技術サポートなど)
- 本番環境の監視とロギングを追加
- ツールの使用状況とパフォーマンスを最適化